# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# 1. Подготовка данных

In [1]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

pd.options.mode.chained_assignment = None

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
data.info()
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Вцелом, с данными всё в порядке, за исключением столбца `Tenure`, здесь есть недостающие значения.

In [3]:
data['Tenure'].unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0., nan])

Видно, что имеются значения `NaN и 0`. В итоге заменю `NaN` на `нули`, так как, скорее всего, отсутствие значения означает отсутствие недвижимости.

In [4]:
data['Tenure'] = data['Tenure'].fillna(0)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Далее мне необходимо закодировать цифрами выраженные в тексте категории, и так как я буду использовать `модель случайного леса` и `дерево решений`, то это `Ordinal Encoding`.

In [5]:
encoder = OrdinalEncoder()
data_ordinal = data[['Surname', 'Geography', 'Gender']]
data_ordinal = pd.DataFrame(encoder.fit_transform(data_ordinal), columns=data_ordinal.columns)
data[['Surname', 'Geography', 'Gender']] = data_ordinal[['Surname', 'Geography', 'Gender']]
display(data.head())
data.info()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,1115.0,619,0.0,0.0,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,1177.0,608,2.0,0.0,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,2040.0,502,0.0,0.0,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,289.0,699,0.0,0.0,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,1822.0,850,2.0,0.0,43,2.0,125510.82,1,1,1,79084.10,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null float64
CreditScore        10000 non-null int64
Geography          10000 non-null float64
Gender             10000 non-null float64
Age                10000 non-null int64
Tenure             10000 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(6), int64(8)
memory usage: 1.1 MB


Закодировались столбцы `'Surname', 'Geography', 'Gender'`

Теперь необходимо разбить данные на 3 выборки.

In [6]:
data_train, data_valid, data_test = np.split(data, [int(.6*len(data)), int(.8*len(data))])

print("Размер обучающей выборки:", data_train.shape)
print("Размер валидационной выборки:", data_valid.shape)
print("Размер тестовой выборки:", data_test.shape)

print('Проверка соотношения значений в столбце "Exited" в исходной таблице: ', data['Exited'].value_counts(normalize=True))
print('Тренировачная выборка:', data_train['Exited'].value_counts(normalize=True))
print('Валидационная выборка:', data_valid['Exited'].value_counts(normalize=True))
print('Тестовая выборка:', data_test['Exited'].value_counts(normalize=True))

Размер обучающей выборки: (6000, 14)
Размер валидационной выборки: (2000, 14)
Размер тестовой выборки: (2000, 14)
Проверка соотношения значений в столбце "Exited" в исходной таблице:  0    0.7963
1    0.2037
Name: Exited, dtype: float64
Тренировачная выборка: 0    0.791833
1    0.208167
Name: Exited, dtype: float64
Валидационная выборка: 0    0.801
1    0.199
Name: Exited, dtype: float64
Тестовая выборка: 0    0.805
1    0.195
Name: Exited, dtype: float64


Видно, что соотношение сохранено.

In [7]:
features_train = data_train.drop(['CustomerId', 'Surname', 'Exited', 'RowNumber'], axis=1)
target_train = data_train['Exited']

features_valid = data_valid.drop(['CustomerId', 'Surname', 'Exited', 'RowNumber'], axis=1)
target_valid = data_valid['Exited']

features_test = data_test.drop(['CustomerId', 'Surname', 'Exited', 'RowNumber'], axis=1)
target_test = data_test['Exited']

Определил, что войдёт в признаки `('Geography', 'Gender', 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary')`, а что в целевой признак `('Exited')`.

Сейчас надо сделать масштабирование признаков.

In [8]:
numeric = ['Geography', 'Gender', 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

display(features_train.head())

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.328324,-0.902929,-1.08202,0.290799,-0.819330,-1.227573,-0.900618,0.644537,0.980850,0.015075
1,-0.441607,1.505952,-1.08202,0.195849,-1.140952,0.116434,-0.900618,-1.551501,0.980850,0.209443
2,-1.533246,-0.902929,-1.08202,0.290799,1.110398,1.332870,2.525956,0.644537,-1.019524,0.233562
3,0.495554,-0.902929,-1.08202,0.005950,-1.140952,-1.227573,0.812669,-1.551501,-1.019524,-0.115541
4,2.050625,1.505952,-1.08202,0.385748,-0.819330,0.785215,-0.900618,0.644537,0.980850,-0.371531


Теперь с данными всё в полном порядке.

# Вывод

* Были удалены 3 столбца из признаков `('CustomerId', 'Surname', 'RowNumber')`
* Дозаполнены недостающие значения
* Данные разбиты на 3 выборки с соотношением ответов в целевом признаке
* Видно сильный дисбаланс классов
* Закодированы столбцы с типом данных `object`
* Выполнено масштабирование всех признаков


Данные подготовлены для дальнейшей работы

# 2. Исследование задачи

Обучаю модель дерева решений и ищу наилучшее значение для depth

In [9]:
max_result = 0
best_depth = 0
for depth in range(1,10):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_valid = model_tree.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    if result > max_result:
        max_result = result
        best_depth = depth
        auc_roc = roc_auc_score(target_valid, predictions_valid)
        
print("F1-score: {0} при max_depth: {1}".format(max_result, best_depth))
print("AUC-ROC:", auc_roc)

F1-score: 0.5678627145085804 при max_depth: 5
AUC-ROC: 0.7096045144574307


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Обучаю модель случайного леса и ищу наилучшее значение для `max_depth`

In [10]:
max_result = 0
best_depth = 0
for depth in range(20, 30):
        model_forest = RandomForestClassifier(n_estimators=20, random_state=12345, max_depth=depth)
        model_forest.fit(features_train, target_train)
        predictions_valid = model_forest.predict(features_valid)
        result = f1_score(target_valid, predictions_valid)
        if result > max_result:
            max_result = result
            best_depth = depth

print("F1-score: {0} при max_depth: {1}".format(max_result, best_depth))

F1-score: 0.5942684766214178 при max_depth: 26


Обучаю модель случайного леса и ищу наилучшее значение для `n_estimators`

In [11]:
max_result = 0
best_est = 0
for est in range(20, 30):
    model_forest = RandomForestClassifier(n_estimators=est, random_state=12345, max_depth=26)
    model_forest.fit(features_train, target_train)
    predictions_valid = model_forest.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    if result > max_result:
        max_result = result
        best_est = est
        auc_roc = roc_auc_score(target_valid, predictions_valid)
        
print("F1-score: {0} при max_depth: {1} и n_estimators: {2}, AUC-ROC: {3}".format(max_result, best_depth, best_est, auc_roc))

F1-score: 0.6074074074074073 при max_depth: 26 и n_estimators: 23, AUC-ROC: 0.7350657783298514


# Вывод

Даже при дисбалансе классов, модель случайного леса показала значение F1 = 0.60 на валидационной выборке, при этом видно, что AUC-ROC и для модели случайного леса и для модели дерева решений находится очень далеко от 1.

# 3. Борьба с дисбалансом

Так как наибольший результат дала модель случайного леса, то дальше буду рассматривать только её.

Первым делом для борьбы с дисбалансом использую `class_weight='balanced'` и нахожу значения для max_depth и n_estimators

In [12]:
max_result = 0
best_depth = 0
for depth in range(10, 20):
    model_forest = RandomForestClassifier(n_estimators=20, random_state=12345, max_depth=depth, class_weight='balanced')
    model_forest.fit(features_train, target_train)
    predictions_valid = model_forest.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    if result > max_result:
        max_result = result
        best_depth = depth
        
print("F1-score: {0} при max_depth: {1}".format(max_result, best_depth))

F1-score: 0.606060606060606 при max_depth: 11


In [13]:
max_result = 0
best_est = 0
for est in range(170, 180):
    model_forest = RandomForestClassifier(n_estimators=est, random_state=12345, max_depth=11, class_weight='balanced')
    model_forest.fit(features_train, target_train)
    predictions_valid = model_forest.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    if result > max_result:
        max_result = result
        best_est = est
        auc_roc = roc_auc_score(target_valid, predictions_valid)
        
print("F1-score: {0} при max_depth: {1} и n_estimators: {2}, AUC-ROC: {3}".format(max_result, best_depth, best_est, auc_roc))

F1-score: 0.6193548387096773 при max_depth: 11 и n_estimators: 173, AUC-ROC: 0.7587484865024247


Можно заметить, что увеличились и `F1 0.607 -> 0.619` и `AUC-ROC 0.735 -> 0.758`, значит данным способом удалось улучшить модель.

Следующий способ - `upsample`.

In [14]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

max_result = 0
best_depth = 0
for depth in range(10, 20):
    model_forest = RandomForestClassifier(n_estimators=20, random_state=12345, max_depth=depth)
    model_forest.fit(features_upsampled, target_upsampled)
    predictions_valid = model_forest.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    if result > max_result:
        max_result = result
        best_depth = depth
        
print("F1-score: {0} при max_depth: {1}".format(max_result, best_depth))

F1-score: 0.6029919447640967 при max_depth: 12


In [15]:
max_result = 0
best_est = 0
for est in range(170, 180):
    model_forest = RandomForestClassifier(n_estimators=est, random_state=12345, max_depth=12)
    model_forest.fit(features_upsampled, target_upsampled)
    predictions_valid = model_forest.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    if result > max_result:
        max_result = result
        best_est = est
        auc_roc = roc_auc_score(target_valid, predictions_valid)
        
print("F1-score: {0} при max_depth: {1} и n_estimators: {2}, AUC-ROC: {3}".format(max_result, best_depth, best_est, auc_roc))

F1-score: 0.6132075471698112 при max_depth: 12 и n_estimators: 172, AUC-ROC: 0.7673322919215303


In [16]:
print("размер выборки до увеличения: {0}, после: {1} ".format(len(target_train), len(target_upsampled)))
print('Соотношение 0 и 1: ')
print(target_upsampled.value_counts(normalize=True))

размер выборки до увеличения: 6000, после: 9747 
Соотношение 0 и 1: 
1    0.512568
0    0.487432
Name: Exited, dtype: float64


Можно заметить, что количество `0 и 1` приравнялось `(0.487 и 0.512)`

`F1` стал больше, чем при дисбалансе `0.607 -> 0.613`, но меньше, чем у предыдущего способа `0.619 -> 0.613`

`AUC-ROC` показывает большее значение, чем в предыдущем методе `0.758 -> 0.767`, это ожидаемо, ведь выборка стала больше `(6000 -> 9747)`, и качество модели улучшилось.

In [17]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.2629)

max_result = 0
best_depth = 0
for depth in range(1, 11):
    model_forest = RandomForestClassifier(n_estimators=20, random_state=12345, max_depth=depth)
    model_forest.fit(features_downsampled, target_downsampled)
    predictions_valid = model_forest.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    if result > max_result:
        max_result = result
        best_depth = depth
        
print("F1-score: {0} при max_depth: {1}".format(max_result, best_depth))

F1-score: 0.5887939221272555 при max_depth: 10


In [18]:
print("размер выборки до увеличения: {0}, после: {1} ".format(len(target_train), len(target_downsampled)))
print('Соотношение 0 и 1: ')
target_downsampled.value_counts(normalize=True)

размер выборки до увеличения: 6000, после: 2498 
Соотношение 0 и 1: 


1    0.5
0    0.5
Name: Exited, dtype: float64

In [19]:
max_result = 0
best_est = 0
for est in range(270, 280):
    model_forest = RandomForestClassifier(n_estimators=est, random_state=12345, max_depth=10)
    model_forest.fit(features_downsampled, target_downsampled)
    predictions_valid = model_forest.predict(features_valid)
    result = f1_score(target_valid, predictions_valid)
    if result > max_result:
        max_result = result
        best_est = est
        auc_roc = roc_auc_score(target_valid, predictions_valid)
        
print("F1-score: {0} при max_depth: {1} и n_estimators: {2}, AUC-ROC: {3}".format(max_result, best_depth, best_est, auc_roc))

F1-score: 0.5980198019801981 при max_depth: 10 и n_estimators: 272, AUC-ROC: 0.7826429274964083


`F1` стал меньше, чем при дисбалансе `0.607 -> 0.598`, а `AUC-ROC` наоборот показало наибольшее значение из всех методов борьбы с дисбалансом, это связано с уменьшение тренировочной выборки `6000 -> 2498`.

# Вывод

Я исследовал 3 способа борьбы с дисбалансом на модели случайного леса и выяснил, что лучший способ - `upsample`, так как в нём F1 и AUC-ROC наиболее сбалансированы.

# 4. Тестирование модели

Тестирование модели случайного леса без учета дисбаланса

In [20]:
model_forest = RandomForestClassifier(n_estimators=23, random_state=12345, max_depth=26)
model_forest.fit(features_train, target_train)
predictions_test = model_forest.predict(features_test)
result = f1_score(target_test, predictions_test)
print("F1: ", result)

F1:  0.5588697017268446


Тестирование модели случайного леса с `class_weight='balanced'`

In [21]:
model_forest = RandomForestClassifier(n_estimators=173, random_state=12345, max_depth=11, class_weight='balanced')
model_forest.fit(features_train, target_train)
predictions_test = model_forest.predict(features_test)
result = f1_score(target_test, predictions_test)
print("F1: ", result)

F1:  0.5898491083676269


Тестирование модели случайного леса с `upsample`

In [22]:
model_forest = RandomForestClassifier(n_estimators=172, random_state=12345, max_depth=12)
model_forest.fit(features_upsampled, target_upsampled)
predictions_test = model_forest.predict(features_test)
result = f1_score(target_test, predictions_test)
print("F1: ", result)

F1:  0.6015037593984962


Тестирование модели случайного леса с `downsample`

In [23]:
model_forest = RandomForestClassifier(n_estimators=272, random_state=12345, max_depth=10)
model_forest.fit(features_downsampled, target_downsampled)
predictions_test = model_forest.predict(features_test)
result = f1_score(target_test, predictions_test)
print("F1: ", result)

F1:  0.5717131474103586


# Вывод

Тестовая выборка всё расставила на свои места:

Минимальное значение `F1` у модели с неучтенным дисбалансом, хотя на валидной выборке она показала высокий результат, качество модели оставляет желать лучшего `(0.558)`

Следующее значение `F1` у модели с `downsample`, так как очень мало данных, то модель недообучилась, отсюда и низкий результат. `(0.571)`

Далее идёт модель с `class_weight='balanced'`, и вот она уже почти попадает под условие со значением `F1 0.589`

И лучшим способом борьбы с дисбалансом оказался `upsample`. Ведь в данном случае самая большая выборка, хоть значения и повторяются. И `F1 0.601`. Таким образом условие получить 0.59 или больше на тестовой выборке выполнено.